# Problems So far ()

1.   

And the way I should approach the task is still a bit vague.


# Google Colab Setup

In [1]:
# ! pip install -q kaggle

In [6]:
# from google.colab import files
# files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amiralism","key":"cfffa868d2456e4a1cb4a6e389c7af4a"}'}

In [7]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [8]:
# ! kaggle competitions download -c 'open-problems-multimodal'

100% 18.1G/18.1G [14:13<00:00, 25.5MB/s]
100% 18.1G/18.1G [14:13<00:00, 22.7MB/s]


In [9]:
# ! mkdir data
# ! unzip open-problems-multimodal.zip -d data

Archive:  open-problems-multimodal.zip
  inflating: data/evaluation_ids.csv  
  inflating: data/metadata.csv       
  inflating: data/metadata_cite_day_2_donor_27678.csv  
  inflating: data/sample_submission.csv  
  inflating: data/test_cite_inputs.h5  
  inflating: data/test_cite_inputs_day_2_donor_27678.h5  
  inflating: data/test_multi_inputs.h5  
  inflating: data/train_cite_inputs.h5  
  inflating: data/train_cite_targets.h5  
  inflating: data/train_multi_inputs.h5  
  inflating: data/train_multi_targets.h5  


In [3]:
# # deepmu package
# ! gdown 1L8S1gpSxheKN9BWlcGE65-rTUdfwCIOm
# ! unzip deepmu.zip

Downloading...
From: https://drive.google.com/uc?id=1L8S1gpSxheKN9BWlcGE65-rTUdfwCIOm
To: /mnt/g/Darsi/Bio/Enformer/deepmu.zip
100%|██████████████████████████████████████| 23.1k/23.1k [00:00<00:00, 27.3MB/s]
Archive:  deepmu.zip
   creating: deepmu/
   creating: deepmu/models/
   creating: deepmu/models/sequence/
  inflating: deepmu/models/sequence/_enformer.py  
  inflating: deepmu/models/sequence/__init__.py  
   creating: deepmu/models/sequence/__pycache__/
  inflating: deepmu/models/sequence/__pycache__/_enformer.cpython-311.pyc  
  inflating: deepmu/models/sequence/__pycache__/__init__.cpython-311.pyc  
  inflating: deepmu/models/__init__.py  
   creating: deepmu/models/__pycache__/
  inflating: deepmu/models/__pycache__/__init__.cpython-311.pyc  
   creating: deepmu/preprocessing/
 extracting: deepmu/preprocessing/_atac.py  
  inflating: deepmu/preprocessing/_genomics.py  
 extracting: deepmu/preprocessing/_rna.py  
  inflating: deepmu/preprocessing/_utils.py  
 extracting: deepm

In [10]:
# ! pip install scanpy --quiet
# ! pip install muon --quiet
# ! pip install mudata --quiet
# ! pip install enformer-pytorch --quiet
# ! pip install biopython --quiet

# Reference

In [ ]:
# ! curl -O https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.6G  100 10.6G    0     0   197M      0  0:00:55  0:00:55 --:--:--  155M


In [ ]:
# ! mkdir reference
# ! tar -xf refdata-gex-GRCh38-2020-A.tar.gz

# Import Libraries

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns
import re
import gc
import warnings
import scanpy as sc
import muon
import deepmu as dm
from tqdm import tqdm
import mudata as md
from mudata import MuData
import anndata as ad
from scipy.sparse import coo_matrix


warnings.filterwarnings('ignore')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION']='python'

/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/uma

# Loading Data

In [3]:
data_dir = "./data_kaggle"

**Some Note**

The lengths of `train_multi_inputs.h5` and `train_multi_targets.h5` are both 105942. 
We sample 5 percent of the whole dataset (almost) randomly and find the peaks and genes that are highly variable.

In [4]:
DATA_LENGTH = 105942

## Loading Meta Data

In [4]:
# loading the meta-data for the whole dataset, including CITE-seq data
df_meta = pd.read_csv(os.path.join(data_dir, "metadata.csv") , index_col='cell_id')
df_meta

,day,donor,cell_type,technology
cell_id,,,,
c2150f55becb,2,27678,HSC,citeseq
65b7edf8a4da,2,27678,HSC,citeseq
c1b26cb1057b,2,27678,EryP,citeseq
917168fa6f83,2,27678,NeuP,citeseq
2b29feeca86d,2,27678,EryP,citeseq
...,...,...,...,...
96a60b026659,10,31800,hidden,multiome
d493e546991e,10,31800,hidden,multiome
05666c99aa48,10,31800,hidden,multiome


In [5]:
# selecting only cells from multiome part
df_meta_mul = df_meta[df_meta.technology == "multiome"].copy()
del df_meta
df_meta_mul.head()

,day,donor,cell_type,technology
cell_id,,,,
458c2ae2c9b1,2,27678,hidden,multiome
01a0659b0710,2,27678,hidden,multiome
028a8bc3f2ba,2,27678,hidden,multiome
7ec0ca8bb863,2,27678,hidden,multiome
caa0b0022cdc,2,27678,hidden,multiome


Multiome Dataset (according to kaggle description)

train/test_multi_inputs.h5 - ATAC-seq peak counts transformed with TF-IDF using the default log(TF) * log(IDF) output (chromatin accessibility), with rows corresponding to cells and columns corresponding to the location of the genome whose level of accessibility is measured, here identified by the genomic coordinates on reference genome GRCh38 provided in the 10x References - 2020-A (July 7, 2020).

train_multi_targets.h5 - RNA gene expression levels as library-size normalized and log1p transformed counts for the same cells.

# Data Preparation

## Selecting rows and columns

### Finding Highly Variable Peaks

In [5]:
np.random.seed(1)
sample_df_atac = pd.DataFrame([])

for i in range(5):
    random_index = np.random.randint(0, DATA_LENGTH-1000)
    df_multi_train_partial = pd.read_hdf(
        os.path.join(data_dir, "train_multi_inputs.h5"), start=random_index, stop=random_index+1000
    )
    sample_df_atac = pd.concat([sample_df_atac, df_multi_train_partial])
    
gc.collect()
sample_df_atac

gene_id,GL000194.1:114519-115365,GL000194.1:55758-56597,GL000194.1:58217-58957,GL000194.1:59535-60431,GL000195.1:119766-120427,GL000195.1:120736-121603,GL000195.1:137437-138345,GL000195.1:15901-16653,GL000195.1:22357-23209,GL000195.1:23751-24619,...,chrY:7722278-7723128,chrY:7723971-7724880,chrY:7729854-7730772,chrY:7731785-7732664,chrY:7810142-7811040,chrY:7814107-7815018,chrY:7818751-7819626,chrY:7836768-7837671,chrY:7869454-7870371,chrY:7873814-7874709
cell_id,,,,,,,,,,,,,,,,,,,,,
9423819a6d94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0e042c38c23f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
b75cb93d4ea7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4a83971f6b90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
901dc731f928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a002713816d9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4a6c3478573e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9e9b116b60d7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Creating AnnData Object
adata_sample_atac = sc.AnnData(sample_df_atac,
    sample_df_atac.index.to_frame(),
    sample_df_atac.columns.to_frame())
adata_sample_atac

AnnData object with n_obs × n_vars = 5000 × 228942
    obs: 'cell_id'
    var: 'gene_id'

In [7]:
sc.pp.normalize_total(adata_sample_atac, target_sum=1e4)
sc.pp.log1p(adata_sample_atac)
sc.pp.highly_variable_genes(adata_sample_atac, n_top_genes=10000)

adata_atac_hvf = adata_sample_atac[:, adata_sample_atac.var.highly_variable].copy()
adata_atac_hvf

AnnData object with n_obs × n_vars = 5000 × 10000
    obs: 'cell_id'
    var: 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [8]:
adata_atac_hvf.var[adata_atac_hvf.var.columns[:1]].to_csv(os.path.join(data_dir, "hv_peaks.csv"))

In [9]:
del adata_atac_hvf, adata_sample_atac, sample_df_atac, df_multi_train_partial
gc.collect()

0

### Finding Highly Variable Genes

In [52]:
np.random.seed(1)
sample_df_rna = pd.DataFrame([])

for i in range(5):
    random_index = np.random.randint(0, DATA_LENGTH-1000)
    df_multi_train_partial = pd.read_hdf(
        os.path.join(data_dir, "train_multi_targets.h5"), start=random_index, stop=random_index+1000
    )
    sample_df_rna = pd.concat([sample_df_rna, df_multi_train_partial])
    
gc.collect()
sample_df_rna

gene_id,ENSG00000121410,ENSG00000268895,ENSG00000175899,ENSG00000245105,ENSG00000166535,ENSG00000256661,ENSG00000184389,ENSG00000128274,ENSG00000094914,ENSG00000081760,...,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755
cell_id,,,,,,,,,,,,,,,,,,,,,
9423819a6d94,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,4.903240,0.0,0.0,5.592669,0.0,0.000000,0.000000,5.592669
0e042c38c23f,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,4.270838,4.956975,...,0.000000,0.000000,4.270838,0.0,0.0,0.000000,0.0,0.000000,0.000000,4.270838
b75cb93d4ea7,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,5.848005
4a83971f6b90,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,5.342478,5.342478,0.0,0.0,5.342478,0.0,0.000000,0.000000,6.033230
901dc731f928,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,6.005802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a002713816d9,0.0,0.000000,4.957466,0.0,0.0,4.271325,0.0,0.0,0.000000,0.000000,...,0.000000,4.957466,0.000000,0.0,0.0,4.957466,0.0,0.000000,0.000000,4.957466
4a6c3478573e,0.0,0.000000,3.900574,0.0,0.0,0.000000,0.0,0.0,3.900574,0.000000,...,0.000000,4.985608,0.000000,0.0,0.0,0.000000,0.0,0.000000,3.900574,4.583555
9e9b116b60d7,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,4.998633,4.312210,0.000000,0.0,0.0,4.998633,0.0,4.998633,4.312210,5.401846


In [53]:
# Creating AnnData Object
adata_sample_rna = sc.AnnData(sample_df_rna,
    sample_df_rna.index.to_frame(),
    sample_df_rna.columns.to_frame())
adata_sample_rna

AnnData object with n_obs × n_vars = 5000 × 23418
    obs: 'cell_id'
    var: 'gene_id'

In [54]:
sc.pp.normalize_total(adata_sample_rna, target_sum=1e4)
sc.pp.log1p(adata_sample_rna)
sc.pp.highly_variable_genes(adata_sample_rna, n_top_genes=2000)

adata_rna_hvf = adata_sample_rna[:, adata_sample_rna.var.highly_variable].copy()
adata_rna_hvf

AnnData object with n_obs × n_vars = 5000 × 2000
    obs: 'cell_id'
    var: 'gene_id', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [55]:
adata_rna_hvf.var[adata_rna_hvf.var.columns[:1]].to_csv(os.path.join(data_dir, "hv_genes.csv"))

In [56]:
del adata_rna_hvf, adata_sample_rna, sample_df_rna, df_multi_train_partial
gc.collect()

34

## Creating the smaller version of data

### ATAC

In [5]:
hv_columns_atac = pd.read_csv(os.path.join(data_dir, "hv_peaks.csv"), index_col=1)
# Selecting only peaks that start with 'chr' 
hv_columns_atac = hv_columns_atac[hv_columns_atac.gene_id.str.startswith("chr")]
hv_columns_atac

,gene_id
gene_id.1,
chr10:100013840-100014756,chr10:100013840-100014756
chr10:100255223-100255983,chr10:100255223-100255983
chr10:100370527-100371417,chr10:100370527-100371417
chr10:100564973-100565808,chr10:100564973-100565808
chr10:100650203-100651061,chr10:100650203-100651061
...,...
chrY:13227302-13228090,chrY:13227302-13228090
chrY:13738366-13739243,chrY:13738366-13739243
chrY:21180640-21181505,chrY:21180640-21181505


In [6]:
hv_columns_rna = pd.read_csv(os.path.join(data_dir, "hv_genes.csv"), index_col=1)
hv_columns_rna

,gene_id
gene_id.1,
ENSG00000242908,ENSG00000242908
ENSG00000115977,ENSG00000115977
ENSG00000205002,ENSG00000205002
ENSG00000183044,ENSG00000183044
ENSG00000154262,ENSG00000154262
...,...
ENSG00000183621,ENSG00000183621
ENSG00000198597,ENSG00000198597
ENSG00000172000,ENSG00000172000


In [7]:
last_indexes = [
    (DATA_LENGTH//4000)*1000, 
    (DATA_LENGTH//4000)*2000, 
    (DATA_LENGTH//4000)*3000, 
    DATA_LENGTH - 1000
]

start_index = 0
for num in range(len(last_indexes)):
    df_atac = pd.DataFrame([])
    df_rna = pd.DataFrame([])
    for i in tqdm(range(start_index, last_indexes[num], 1000)):
        if last_indexes[num] - i >= 1000:
            # ATAC
            df_multi_train_partial = pd.read_hdf(
                os.path.join(data_dir, "train_multi_inputs.h5"), start=i, stop=i+1000
            )
            df_multi_train_partial = df_multi_train_partial[hv_columns_atac.gene_id.values].astype(pd.SparseDtype(float, 0))
            df_atac = pd.concat([df_atac, df_multi_train_partial])
            # RNA
            df_multi_train_partial = pd.read_hdf(
                os.path.join(data_dir, "train_multi_targets.h5"), start=i, stop=i+1000
            )
            df_multi_train_partial = df_multi_train_partial[hv_columns_rna.gene_id.values].astype(pd.SparseDtype(float, 0))
            df_rna = pd.concat([df_rna, df_multi_train_partial])
            
        else: ##################
            # ATAC
            df_multi_train_partial = pd.read_hdf(
                os.path.join(data_dir, "train_multi_inputs.h5"), start=i, stop=DATA_LENGTH
            )
            df_multi_train_partial = df_multi_train_partial[hv_columns_atac.gene_id.values].astype(pd.SparseDtype(float, 0))
            df_atac = pd.concat([df_atac, df_multi_train_partial])
            # RNA
            df_multi_train_partial = pd.read_hdf(
                os.path.join(data_dir, "train_multi_targets.h5"), start=i, stop=DATA_LENGTH
            )
            df_multi_train_partial = df_multi_train_partial[hv_columns_rna.gene_id.values].astype(pd.SparseDtype(float, 0))
            df_rna = pd.concat([df_rna, df_multi_train_partial])  
        gc.collect()

        
    start_index = last_indexes[num]
    
    adata_atac = sc.AnnData(df_atac.replace(np.nan, 0),
        df_atac.index.to_frame(),
        df_atac.columns.to_frame())
    
    adata_rna = sc.AnnData(df_rna.replace(np.nan, 0),
        df_rna.index.to_frame(),
        df_rna.columns.to_frame())
    
    mdata = MuData({"atac": adata_atac, "rna": adata_rna})
    
    mdata.write(
        os.path.join(data_dir, f"mudata_part_{num+1}.h5mu")
    )

100%|███████████████████████████████████████████████████████████████████████████████████| 27/27 [04:24<00:00,  9.78s/it]


## Cheking the .h5mu files we created

In [8]:
mdata1 = md.read(
    os.path.join(data_dir, f"mudata_part_1.h5mu")
)
mdata2 = md.read(
    os.path.join(data_dir, f"mudata_part_2.h5mu")
)
mdata3 = md.read(
    os.path.join(data_dir, f"mudata_part_3.h5mu")
)
mdata4 = md.read(
    os.path.join(data_dir, f"mudata_part_4.h5mu")
)

In [9]:
mdata1.shape, mdata2.shape, mdata3.shape, mdata4.shape

((26000, 11997), (26000, 11997), (26000, 11997), (27942, 11997))

In [10]:
mdata1.var

,gene_id
gene_id,
chr10:100013840-100014756,chr10:100013840-100014756
chr10:100255223-100255983,chr10:100255223-100255983
chr10:100370527-100371417,chr10:100370527-100371417
chr10:100564973-100565808,chr10:100564973-100565808
chr10:100650203-100651061,chr10:100650203-100651061
...,...
ENSG00000183621,ENSG00000183621
ENSG00000198597,ENSG00000198597
ENSG00000172000,ENSG00000172000


## Adding Coordinates

In [11]:
def get_gene_coordinates(adata, av_genes):
    adata_ = adata[:, list(av_genes)]
    adata_.var = pd.concat([adata_.var, gene_coordinates.loc[av_genes, ("chr","start", "end")]], axis=1)
    return adata_

In [12]:
def extract_atac_coordinates_(serie):
    df = serie.str.split(pat='[-:]', expand=True)
    return df.rename(columns={0:"chr", 1:"start", 2:"end"})

def get_atac_coordinates(adata):
    adata.var = pd.concat(
        [adata.var, extract_atac_coordinates_(adata.var["gene_id"])],
        axis=1
    )
    return adata

### RNA

In [13]:
# This DataFrame is exported from deepmu package
gene_coordinates = pd.read_csv(
    os.path.join(data_dir, "gene_coordinates.csv"),
    index_col=0
)
gene_coordinates.head(2)

,feature_types,gene_id,index,Ensembl GeneIDs,Gene Type,Synonyms,Symbol,Description,seqname,start,end,chr
ENSG00000188976,GEX,ENSG00000188976,NOC2L,ENSG00000188976,PROTEIN_CODING,"NIR,NET7,NET15,PPP1R112",NOC2L,NOC2 like nucleolar associated transcriptional...,chr1,944697,959240,chr1
ENSG00000187961,GEX,ENSG00000187961,KLHL17,ENSG00000187961,PROTEIN_CODING,AF,KLHL17,kelch like family member 17,chr1,960694,965188,chr1


**Important Note**

I didn't know where to find the coordinates of all genes, so I only use the ones whose coordinates were available in `deepmu` package.

In [14]:
available_genes = set(gene_coordinates.index).intersection(
    set(mdata1["rna"].var.gene_id.values)
)

## Creating the Final AnnData Objects

In [15]:
mdatas = [mdata1, mdata2, mdata3, mdata4]

In [16]:
rna_anndata = ad.concat([
    mdata['rna'] for mdata in mdatas
], merge="same")

atac_anndata = ad.concat([
    mdata['atac'] for mdata in mdatas
], merge="same")

In [17]:
rna_anndata = get_gene_coordinates(rna_anndata, available_genes)
atac_anndata = get_atac_coordinates(atac_anndata)

In [18]:
atac_anndata.write_h5ad(
    os.path.join(data_dir, "atac_anndata.h5ad")
)
rna_anndata.write_h5ad(
    os.path.join(data_dir, "rna_anndata.h5ad")
)

# Extracting Embeddings of Sequences Using Enformer

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
###############################################
###############################################
###############################################
###############################################

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns
import re
import gc
import warnings
import scanpy as sc
import muon
import deepmu as dm
from tqdm import tqdm
import mudata as md
from mudata import MuData
import anndata as ad
from scipy.sparse import coo_matrix


warnings.filterwarnings('ignore')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION']='python'


data_dir = "./data_kaggle"
DATA_LENGTH = 105942

/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/uma

## Loading the Dataset and the Enformer Model

In [3]:
atac_anndata = ad.read_h5ad(
    os.path.join(data_dir, "atac_anndata.h5ad")
)

rna_anndata = ad.read_h5ad(
    os.path.join(data_dir, "rna_anndata.h5ad")
)
mdata = MuData({"atac": atac_anndata, "rna": rna_anndata})
mdata

MuData object with n_obs × n_vars = 105942 × 10972
  var:	'gene_id', 'chr', 'start', 'end'
  2 modalities
    atac:	105942 x 9997
      obs:	'cell_id'
      var:	'gene_id', 'chr', 'start', 'end'
    rna:	105942 x 975
      obs:	'cell_id'
      var:	'gene_id', 'chr', 'start', 'end'

In [4]:
mdata["atac"].var = mdata["atac"].var.astype({
    "start": int,
    "end": int,
})

mdata["rna"].var = mdata["rna"].var.astype({
    "start": int,
    "end": int,
})

In [5]:
enformer = dm.models.seq.EnformerWrapper()

File already exists...


455it [05:27,  1.39it/s] 


## Extracting Embeddings

### ATAC

In [ ]:
enformer.extract_embeddings_from_mudata(mdata, 
                                        modality='atac', 
                                        window_size='max', 
                                        agg_method='sum',
                                        )

 46%|███████████████████████████████████▏                                         | 4574/9997 [52:08<1:04:37,  1.40it/s]

In [17]:
mdata["atac"].varm["enformer_emb"]

array([[ 1.16696060e+02,  5.33300066e+00,  4.36163879e+02, ...,
        -2.61828482e-01,  7.97865417e+02,  2.97136765e+01],
       [-7.59147720e+01, -7.48912573e-01,  3.03640175e+01, ...,
        -1.08701713e-01, -1.09746269e+02,  3.05580330e+01],
       [ 7.45069580e+01, -1.22915030e+00,  2.16997314e+02, ...,
        -5.73890448e-01,  3.08295441e+02,  7.72889481e+01],
       ...,
       [-8.90504532e+01, -2.85261452e-01, -9.70807343e+01, ...,
        -3.60570192e-01, -1.06433640e+02, -6.74439774e+01],
       [ 4.73809910e+00, -1.73017025e+00, -1.21149712e+02, ...,
        -2.49075085e-01,  4.16925583e+01, -1.44388924e+01],
       [-1.14436325e+02, -9.25791413e-02, -1.24260132e+02, ...,
        -1.94091618e-01, -1.00297905e+02, -3.91020088e+01]], dtype=float32)

In [19]:
# # Saving the embeddings
# with open(os.path.join(data_dir, 'atac_embd.npy'), 'wb') as f:
#     np.save(f, mdata["atac"].varm["enformer_emb"])

### RNA

In [21]:
enformer.extract_embeddings_from_mudata(mdata, 
                                        modality='rna', 
                                        window_size='max', 
                                        agg_method='sum',
                                        )

100%|█████████████████████████████████████████████████████████████████████████████████| 975/975 [10:41<00:00,  1.52it/s]


In [22]:
mdata["rna"].varm["enformer_emb"]

array([[-9.14816818e+01, -3.92256141e-01, -1.25419861e+02, ...,
        -4.72265065e-01,  1.64335495e+02,  2.98313313e+01],
       [-6.12116814e+01, -8.94101858e-01, -1.26663773e+02, ...,
        -1.38199165e-01, -8.17191467e+01,  9.18119812e+00],
       [ 2.15720398e+02,  2.44957492e-01, -1.13366837e+02, ...,
        -7.73207992e-02, -1.01635025e+02,  7.18337936e+01],
       ...,
       [ 4.36805969e+02,  1.03119507e+01, -9.44195023e+01, ...,
        -7.77994692e-02, -6.95453491e+01,  3.57042618e+01],
       [-9.85135651e+01, -3.14075559e-01, -1.30791489e+02, ...,
        -1.08284377e-01, -1.04351120e+02, -1.49595518e+01],
       [-1.14406219e+02,  5.93711257e-01, -3.98234978e+01, ...,
        -1.17217112e+00, -1.02825958e+02, -6.07167587e+01]], dtype=float32)

In [23]:
# # Saving the embeddings
# with open(os.path.join(data_dir, 'rna_embd.npy'), 'wb') as f:
#     np.save(f, mdata["rna"].varm["enformer_emb"])

# Training The Model

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from torch import nn
from torch import Tensor
from torch.nn import Module
from torch.nn import Linear
from torch.nn import ReLU, GELU, Sigmoid 
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [3]:
###############################################
###############################################
###############################################
###############################################

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns
import re
import gc
import warnings
import scanpy as sc
import muon
import deepmu as dm
from tqdm import tqdm
import mudata as md
from mudata import MuData
import anndata as ad
from scipy.sparse import coo_matrix


warnings.filterwarnings('ignore')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION']='python'


data_dir = "./data_kaggle"
DATA_LENGTH = 105942

/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/uma

## Loading the Dataset 

In [4]:
atac_anndata = ad.read_h5ad(
    os.path.join(data_dir, "atac_anndata.h5ad")
)

rna_anndata = ad.read_h5ad(
    os.path.join(data_dir, "rna_anndata.h5ad")
)
mdata = MuData({"atac": atac_anndata, "rna": rna_anndata})

mdata["atac"].var = mdata["atac"].var.astype({
    "start": int,
    "end": int,
})

mdata["rna"].var = mdata["rna"].var.astype({
    "start": int,
    "end": int,
})

mdata

MuData object with n_obs × n_vars = 105942 × 10972
  var:	'gene_id', 'chr', 'start', 'end'
  2 modalities
    atac:	105942 x 9997
      obs:	'cell_id'
      var:	'gene_id', 'chr', 'start', 'end'
    rna:	105942 x 975
      obs:	'cell_id'
      var:	'gene_id', 'chr', 'start', 'end'

## Loading the Extracted Embeddings

In [5]:
with open(os.path.join(data_dir, 'atac_embd.npy'), 'rb') as f:
    atac_embd = np.load(f)
with open(os.path.join(data_dir, 'rna_embd.npy'), 'rb') as f:
    rna_embd = np.load(f)

In [6]:
atac_embd.shape, rna_embd.shape

((9997, 3072), (975, 3072))

In [7]:
ENFORMER_EMBEDDING_SIZE = atac_embd.shape[1]
ATAC_SIZE = atac_embd.shape[0]
RNA_SIZE = rna_embd.shape[0]
ENFORMER_EMBEDDING_SIZE, ATAC_SIZE, RNA_SIZE

(3072, 9997, 975)

## Preprocessing

### RNA

Checking if the RNA-seq Data is 0-1 normalized

In [8]:
(mdata["rna"].X).max(), (mdata["rna"].X).min(), (mdata["rna"].X).mean()

(12.134982109069824, 0.0, 1.3299917939861507)

In [9]:
(mdata["rna"].X).sum(axis=1)

matrix([[1075.20572233],
        [ 641.77443409],
        [1090.19545174],
        ...,
        [1680.57458687],
        [ 945.07147026],
        [1273.00603914]])

It's not normalized, so we normalize it.

In [10]:
mdata["rna"].layers["normalized"] = mdata["rna"].X / (mdata["rna"].X).max() # should have been sc.pp.normalize_total(mdata["rna"], target_sum=30, inplace=False)['X']
mdata["rna"].layers["normalized"]

<105942x975 sparse matrix of type '<class 'numpy.float64'>'
	with 26171996 stored elements in Compressed Sparse Row format>

The reason why I didn't use `sc.pp.normalize_total` was that it would shrink numbers to very small numbers (on average 0.03) and it would lead to the problem of vanishing gradient.

In [11]:
mdata["rna"].layers["normalized"].max(), mdata["rna"].layers["normalized"].min(), mdata["rna"].layers["normalized"].mean()

(1.0, 0.0, 0.10959981498383091)

### ATAC

Checking if the ATAC-seq Data is normalized

In [12]:
(mdata["atac"].X).max(), (mdata["atac"].X).min(), (mdata["atac"].X).mean()

(17.857282638549805, 0.0, 0.034681766695024416)

In [13]:
(mdata["atac"].X).sum(axis=1)

matrix([[477.37722731],
        [288.90541077],
        [357.80580312],
        ...,
        [381.34842807],
        [415.95030707],
        [256.66030061]])

It's not normalized, so we normalize it.

In [14]:
mdata["atac"].layers["normalized"] = mdata["atac"].X / (mdata["atac"].X).max() #sc.pp.normalize_total(mdata["atac"], target_sum=2.9, inplace=False)['X']
mdata["atac"].layers["normalized"]

<105942x9997 sparse matrix of type '<class 'numpy.float64'>'
	with 15961781 stored elements in Compressed Sparse Row format>

The reason why I didn't use `sc.pp.normalize_total` was that it would shrink numbers to very small numbers (on average 0.0002) and it would lead to the problem of vanishing gradient.

In [15]:
mdata["atac"].layers["normalized"].max(), mdata["atac"].layers["normalized"].min(), mdata["atac"].layers["normalized"].mean()

(1.0, 0.0, 0.001942163732132143)

## Defining the Model

We use a MLP model in order to predict chromatin accessibility and gene expression.

In [16]:
class Emb2Cell(nn.Module):
    def __init__(self, embedding_size, num_cells, hidden_layers=None, device=device):
        super().__init__()
        if hidden_layers==None:
            hidden_layers = [512, 64, 16]
        
        self.layers = [Linear(num_cells + embedding_size, hidden_layers[0], device=device)] + [
            Linear(hidden_layers[i], hidden_layers[i + 1], device=device) for i in range(len(hidden_layers)-1)
        ] + [Linear(hidden_layers[-1], 2, device=device)]
        
        self.activations = [GELU() for i in range(len(hidden_layers))]
        self.batch_norms = [nn.BatchNorm1d(hidden_layers[i], device=device) for i in range(len(hidden_layers))]
        self.dropouts = [nn.Dropout(0.3, inplace=True) for i in range(len(hidden_layers))]
        
        
        for i in range(len(self.layers)):
            setattr(self, f'FCLayer_{i+1}', self.layers[i])
            
        for i in range(len(self.activations)):
            setattr(self, f'ActivationLayer_{i+1}', self.activations[i])
            
        for i in range(len(self.batch_norms)):
            setattr(self, f'BatchNorm_{i+1}', self.batch_norms[i])
            
        for i in range(len(self.dropouts)):
            setattr(self, f'Drpout_{i+1}', self.dropouts[i])
            
    def forward(self, X):
        current = X
        for i, layer_ in enumerate(self.layers):
            if i == len(self.layers)-1:
                current = layer_(current)
            else:
                current = layer_(current)
                current = self.batch_norms[i](current)
                current = self.activations[i](current)
                current = self.dropouts[i](current)
        return current
            
model = Emb2Cell(ENFORMER_EMBEDDING_SIZE, DATA_LENGTH)    

We define a helper function that converts sparse arrays into dense torch tensors.

In [17]:
def get_tensor_from_sparse(sparse_array, device=device):
    coo = coo_matrix(sparse_array)
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    return torch.sparse.FloatTensor(i, v, torch.Size(coo.shape)).to(device).to_dense()

In [18]:
# Model Test
batch_size = 10
X_test = get_tensor_from_sparse(mdata["atac"].layers["normalized"][:, :batch_size]).reshape((batch_size, -1))
inp = torch.cat([torch.randn(batch_size, ENFORMER_EMBEDDING_SIZE, device=device), X_test], dim=1)
model(inp)

tensor([[ 0.2361,  0.7849],
        [ 0.5639,  0.0730],
        [ 0.0730,  0.7732],
        [ 0.0257, -0.3636],
        [-0.3150,  0.6528],
        [-0.2383,  0.4834],
        [-0.0710, -0.0058],
        [-0.1616, -0.0528],
        [-0.4661,  0.6912],
        [-0.2798,  0.1131]], device='cuda:0', grad_fn=<AddmmBackward0>)

## Defining the DataLoader

To make our `Dataset` objects able to use a specific subset of genes/peaks or cells for training while keeping the rest for the test set, we need to pass 2 pandas DataFrame objects to their constructor function. The values of 1 in column use means that this peak/gene/cell will be present in the resulting `Dataset` object.

In [19]:
# This DataFrame indicates which cells are going to be used in dataloader
subset_cells = mdata["rna"].obs.copy().reset_index(drop=True)
subset_cells["use"] = 1
subset_cells

,cell_id,use
0,56390cf1b95e,1
1,fc0c60183c33,1
2,9b4a87e22ad0,1
3,81cccad8cd81,1
4,15cb3d85c232,1
...,...,...
105937,063cead1a4ea,1
105938,553bca99ba78,1
105939,00783f28b463,1
105940,e7abb1a0f251,1


In [20]:
# This DataFrame indicates which genes are going to be used in dataloader
subset_genes = mdata["rna"].var.copy().drop(["start", "end", "chr"], axis=1).reset_index(drop=True)
subset_genes["use"] = 1
subset_genes

,gene_id,use
0,ENSG00000030582,1
1,ENSG00000139970,1
2,ENSG00000172159,1
3,ENSG00000039319,1
4,ENSG00000154102,1
...,...,...
970,ENSG00000110066,1
971,ENSG00000100055,1
972,ENSG00000107968,1
973,ENSG00000182732,1


In [21]:
# This DataFrame indicates which peaks are going to be used in dataloader
subset_peaks = mdata["atac"].var.copy().drop(["start", "end", "chr"], axis=1).reset_index(drop=True)
subset_peaks["use"] = 1
subset_peaks

,gene_id,use
0,chr10:100013840-100014756,1
1,chr10:100255223-100255983,1
2,chr10:100370527-100371417,1
3,chr10:100564973-100565808,1
4,chr10:100650203-100651061,1
...,...,...
9992,chrY:13227302-13228090,1
9993,chrY:13738366-13739243,1
9994,chrY:21180640-21181505,1
9995,chrY:6908816-6909637,1


### RNA

In [22]:
class RNADataset(Dataset):
    def __init__(self, subset_genes, subset_cells, device=device):
        super(RNADataset, self).__init__()
        
        self.subset_genes_used = subset_genes[subset_genes.use==1].copy()
        self.subset_cells_used = subset_cells[subset_cells.use==1].copy()

        self.cell_size = len(self.subset_cells_used)
        self.rna_size = len(self.subset_genes_used)
        self.device = device

    def __len__(self):
        return self.rna_size * self.cell_size

    def __getitem__(self, idx):
        cell_id = idx // self.rna_size
        cell_id = self.subset_cells_used.index[cell_id]
        
        gene_id = idx % self.rna_size
        gene_id = self.subset_genes_used.index[gene_id]
        
        
        embd = torch.tensor(rna_embd[gene_id], device=self.device)
        one_hot = torch.zeros(DATA_LENGTH, dtype=torch.int, device=self.device).scatter_(
            dim=0, 
            index=torch.tensor(cell_id, device=self.device), 
            value=1
        )
        X = torch.cat([embd, one_hot])
        y = get_tensor_from_sparse(mdata["rna"].layers["normalized"][cell_id, gene_id], device=self.device).reshape(1)
        return X, y

Now we test the data loader:

In [23]:
subset_genes.loc[10:,"use"] = 0
subset_cells.loc[10:,"use"] = 0

rna_data = RNADataset(subset_genes, subset_cells)
rna_data = DataLoader(rna_data, batch_size=10, shuffle=True)

In [24]:
for X, y in tqdm(rna_data):
    print(X.shape, y.shape)
    pass

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 124.80it/s]

torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])


### ATAC

In [25]:
class ATACDataset(Dataset):
    def __init__(self, subset_peaks, subset_cells, device=device):
        super(ATACDataset, self).__init__()
        
        self.subset_peaks_used = subset_peaks[subset_peaks.use==1].copy()
        self.subset_cells_used = subset_cells[subset_cells.use==1].copy()

        self.cell_size = len(self.subset_cells_used)
        self.peaks_size = len(self.subset_peaks_used)
        self.device = device

    def __len__(self):
        return self.peaks_size * self.cell_size

    def __getitem__(self, idx):
        cell_id = idx // self.peaks_size
        cell_id = self.subset_cells_used.index[cell_id]
        
        peak_id = idx % self.peaks_size
        peak_id = self.subset_peaks_used.index[peak_id]
        
        embd = torch.tensor(atac_embd[peak_id], device=self.device)
        one_hot = torch.zeros(DATA_LENGTH, dtype=torch.int, device=self.device).scatter_(
            dim=0, 
            index=torch.tensor(cell_id, device=self.device), 
            value=1
        )
        X = torch.cat([embd, one_hot])
        y = get_tensor_from_sparse(mdata["atac"].layers["normalized"][cell_id, peak_id], device=self.device).reshape(1)
        return X, y

Now we test the data loader:

In [26]:
subset_peaks.loc[10:,"use"] = 0
subset_cells.loc[10:,"use"] = 0

atac_data = ATACDataset(subset_peaks, subset_cells)
atac_data = DataLoader(atac_data, batch_size=10, shuffle=True)

In [27]:
for X, y in tqdm(atac_data):
    print(X.shape, y.shape)
    pass

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 154.34it/s]

torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])
torch.Size([10, 109014]) torch.Size([10, 1])


## Training Loop

Specifying training variables:

In [28]:
n_epochs = 50
criterion = BCEWithLogitsLoss()
learning_rate = 0.001
optimizer = Adam(model.parameters(), lr=learning_rate)
batch_size = 64

In [29]:
# just optimizing for first 5000 cells, all peaks and first 1500 genes
subset_cells.loc[:,"use"] = 0
subset_cells.loc[:5000,"use"] = 1

subset_genes.loc[:,"use"] = 0
subset_genes.loc[:1500,"use"] = 1

subset_peaks.loc[:,"use"] = 0
subset_peaks.loc[:,"use"] = 1

rna_dataloader = DataLoader(
    RNADataset(subset_genes, subset_cells),
    batch_size=batch_size,
    shuffle=True
)

atac_dataloader = DataLoader(
    ATACDataset(subset_peaks, subset_cells),
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
for epoch in range(n_epochs):
    ### RNA
    
    for X_batch, y_batch in tqdm(rna_dataloader, desc=f"RNA-epoch {epoch+1}"):
        y_pred = model(X_batch)[:, 0:1]
        loss_rna = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss_rna.backward()
        optimizer.step()
        
    for X_batch, y_batch in tqdm(atac_dataloader, desc=f"ATAC-epoch {epoch+1}"):
        y_pred = model(X_batch)[:, 1:]
        loss_atac = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss_atac.backward()
        optimizer.step()
        
    
        

RNA-epoch 1:   0%|                                                                | 125/76188 [00:15<2:31:48,  8.35it/s]